1. Использовать датасет MovieLens.  
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:  
- TF-IDF на тегах и жанрах;  
- средние оценки (+ median, variance и т. д.) пользователя и фильма.  
3. Оценить RMSE на тестовой выборке.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [152]:
print(movies.shape)
movies.head()

(9742, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [153]:
print(tags.shape)
tags.head()

(3683, 4)


,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [154]:
print(ratings.shape)
ratings.head()

(100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### TAGS

In [6]:
tags_grouped = tags.groupby('movieId').count().reset_index()

In [7]:
tags_grouped.drop(columns=['userId', 'timestamp'], inplace=True)

In [8]:
tags_grouped

,movieId,tag
0,1,3
1,2,4
2,3,2
3,5,2
4,7,1
...,...,...
1567,183611,3
1568,184471,3
1569,187593,3
1570,187595,2


In [9]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_strings = []

for movie, group in tags.groupby('movieId'):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))

In [10]:
tfidf_tag = TfidfVectorizer()
X_train_tfidf_tag = tfidf_tag.fit_transform(tag_strings)
X_train_tfidf_tag

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [11]:
tags_tfidf = pd.DataFrame(X_train_tfidf_tag.toarray(), columns=tfidf_tag.get_feature_names())

In [12]:
tags_tfidf

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
tags_tfidf = tags_grouped.join(tags_tfidf, how='left')

In [14]:
tags_tfidf

,movieId,tag,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,183611,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1568,184471,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1569,187593,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,187595,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### MOVIES

In [15]:
def change_string_(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [16]:
movies['genres'].values #!!!!!!!

array(['Adventure|Animation|Children|Comedy|Fantasy',
       'Adventure|Children|Fantasy', 'Comedy|Romance', ..., 'Drama',
       'Action|Animation', 'Comedy'], dtype=object)

In [17]:
movie_genres = [change_string_(g) for g in movies.genres.values]
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [18]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [19]:
movies_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names())

In [20]:
movies_tfidf

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [21]:
movies_tfidf = movies.join(movies_tfidf, how='left')

In [22]:
movies_tfidf

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,193585,Flint (2017),Drama,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


### MOVIES-TAGS

In [23]:
movies_tags = movies_tfidf.merge(tags_tfidf, how='left')

In [38]:
movies_tags = movies_tags.fillna(0)

In [39]:
movies_tags

,movieId,title,genres,action,adventure,animation,children,comedy,crime,documentary,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9739,193585,Flint (2017),Drama,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
list(movies_tags)

['movieId',
 'title',
 'genres',
 'action',
 'adventure',
 'animation',
 'children',
 'comedy',
 'crime',
 'documentary',
 'drama',
 'fantasy',
 'filmnoir',
 'horror',
 'imax',
 'musical',
 'mystery',
 'nogenreslisted',
 'romance',
 'scifi',
 'thriller',
 'war',
 'western',
 'tag',
 '06oscarnominatedbestmovieanimation',
 '1900s',
 '1920s',
 '1950s',
 '1960s',
 '1970s',
 '1980s',
 '1990s',
 '2001like',
 '2danimation',
 '70mm',
 '80',
 'aardman',
 'abortion',
 'absorbing',
 'abstract',
 'academyaward',
 'accident',
 'achronological',
 'acleverchefrat',
 'acting',
 'actionchoreography',
 'actionpacked',
 'adamsandler',
 'addiction',
 'adingoatemybaby',
 'adolescence',
 'adoption',
 'adorable',
 'adrienbrody',
 'adultery',
 'adulthumor',
 'afghanistan',
 'africa',
 'agathachristie',
 'aggressive',
 'aging',
 'aids',
 'alanrickman',
 'alcatraz',
 'alcoholism',
 'alfredhitchcock',
 'aliciavikander',
 'aliens',
 'allegorical',
 'aloneintheworld',
 'alpacino',
 'alterego',
 'alternateendings',

### RATINGS

In [26]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [27]:
ratings_groupedby_movies = ratings.groupby('movieId').mean().reset_index()

In [28]:
ratings_groupedby_movies.drop(columns=['userId', 'timestamp'], inplace=True)

In [29]:
ratings_groupedby_movies.rename(columns={'rating':'movies_mean_rating'}, inplace=True)

In [30]:
ratings_groupedby_movies

,movieId,movies_mean_rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429
...,...,...
9719,193581,4.000000
9720,193583,3.500000
9721,193585,3.500000
9722,193587,3.500000


In [31]:
ratings_groupedby_users = ratings.groupby('userId').mean().reset_index()

In [32]:
ratings_groupedby_users.drop(columns=['movieId', 'timestamp'], inplace=True)

In [33]:
ratings_groupedby_users.rename(columns={'rating':'users_mean_rating'}, inplace=True)

In [34]:
ratings_groupedby_users

,userId,users_mean_rating
0,1,4.366379
1,2,3.948276
2,3,2.435897
3,4,3.555556
4,5,3.636364
...,...,...
605,606,3.657399
606,607,3.786096
607,608,3.134176
608,609,3.270270


### MERGE ALL

In [35]:
ratings_mean_rat = ratings.merge(ratings_groupedby_users, on='userId', how='left')

In [36]:
ratings_mean_rat

,userId,movieId,rating,timestamp,users_mean_rating
0,1,1,4.0,964982703,4.366379
1,1,3,4.0,964981247,4.366379
2,1,6,4.0,964982224,4.366379
3,1,47,5.0,964983815,4.366379
4,1,50,5.0,964982931,4.366379
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.688556
100832,610,168248,5.0,1493850091,3.688556
100833,610,168250,5.0,1494273047,3.688556
100834,610,168252,5.0,1493846352,3.688556


In [40]:
movies_mean_rat = ratings_groupedby_movies.merge(movies_tags, on='movieId', how='left')

In [41]:
movies_mean_rat

,movieId,movies_mean_rating,title,genres,action,adventure,animation,children,comedy,crime,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,3.920930,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,3.431818,Jumanji (1995),Adventure|Children|Fantasy,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,3.259615,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2.357143,Waiting to Exhale (1995),Comedy|Drama|Romance,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,3.071429,Father of the Bride Part II (1995),Comedy,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,193581,4.000000,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9720,193583,3.500000,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9721,193585,3.500000,Flint (2017),Drama,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9722,193587,3.500000,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df = ratings_mean_rat.merge(movies_mean_rat, on='movieId', how='left')

In [45]:
df.drop(columns=['timestamp', 'title', 'genres'], inplace=True)

In [46]:
df

,userId,movieId,rating,users_mean_rating,movies_mean_rating,action,adventure,animation,children,comedy,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,1,1,4.0,4.366379,3.920930,0.000000,0.416846,0.516225,0.504845,0.267586,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,4.0,4.366379,3.259615,0.000000,0.000000,0.000000,0.000000,0.570915,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,6,4.0,4.366379,3.946078,0.549328,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,47,5.0,4.366379,3.975369,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,50,5.0,4.366379,4.237745,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,610,166534,4.0,3.688556,3.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100832,610,168248,5.0,3.688556,4.142857,0.549328,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100833,610,168250,5.0,3.688556,3.633333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100834,610,168252,5.0,3.688556,4.280000,0.629882,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
y = df['rating']

In [155]:
X = df.drop(columns=['userId', 'movieId', 'rating'])

In [156]:
X.head()

,users_mean_rating,movies_mean_rating,action,adventure,animation,children,comedy,crime,documentary,drama,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,4.366379,3.920930,0.000000,0.416846,0.516225,0.504845,0.267586,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.366379,3.259615,0.000000,0.000000,0.000000,0.000000,0.570915,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.366379,3.946078,0.549328,0.000000,0.000000,0.000000,0.000000,0.635947,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.366379,3.975369,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.366379,4.237745,0.000000,0.000000,0.000000,0.000000,0.000000,0.553854,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### LINEAR REGRESSION

In [51]:
from sklearn.model_selection import train_test_split

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [58]:
from sklearn.linear_model import LinearRegression

In [161]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [166]:
y_pred = model.predict(X_test)

In [167]:
y_pred

array([3.296778  , 3.38047377, 2.62192845, ..., 3.68229112, 3.36785847,
       2.09296338])

In [168]:
def norm_(i): # приведение предсказанных значений к значениям рейтингов
        
    if 0<=i<0.5:
        i = 0
            
    elif 0.5<=i<0.75:
        i = 0.5
    elif 0.75<=i<1.25:
        i = 1
    elif 1.25<=i<1.75:
        i = 1.5
    elif 1.75<=i<2.25:
        i = 2
    elif 2.25<=i<2.75:
        i = 2.5
    elif 2.75<=i<3.25:
        i = 3
    elif 3.25<=i<3.75:
        i=3.5
    elif 3.75<=i<4.25:
        i = 4
    elif 4.25<=i<4.75:
        i = 4.5
    elif i>=4.75:
        i = 5
    return i

In [169]:
y_pred_n =[]

for i in y_pred:
        
    y_pred_n.append(norm_(i))

### RMSE

In [63]:
from sklearn.metrics import mean_squared_error

In [170]:
mean_squared_error(y_test, y_pred_n, squared=False)

0.8220480161009359